In [1]:
import os
import subprocess
import pandas as pd


os.environ["CUDA_VISIBLE_DEVICES"] ="0"



pythoner = "/home/sxh/anaconda3/envs/chemprop/bin/python"
optimizer = "/home/sxh/Research/chemprop/hyperparameter_optimization.py"
trainer = "/home/sxh/Research/chemprop/train.py"
predicter = "/home/sxh/Research/chemprop/predict.py"


data_path = '/home/sxh/Research/bidd-molmap/paper/07_BACE_HPs/data'
work_path = '/home/sxh/Research/bidd-molmap/paper/07_BACE_HPs/02_predict_DMPNN_attentiveFP'


tr_path = os.path.join(data_path,'train.csv')
vl_path = os.path.join(data_path,'val.csv')
ts_path = os.path.join(data_path,'test.csv')


drug_path  = os.path.join(data_path,'drugs.csv')
df_drug = pd.read_csv(drug_path).set_index('smiles')
df_drug['Class'] = 1
df_drug[['Class']].to_csv(drug_path + '.dmpnn.csv') #it needs smiles col is the first column
drug_path = drug_path + '.dmpnn.csv'


novel_path  = os.path.join(data_path,'bace_chembl_novel.csv')
pd.read_csv(novel_path).set_index('smiles')[['Class']].to_csv(novel_path + '.dmpnn.csv') #it needs smiles col is the first column
novel_path = novel_path + '.dmpnn.csv'


common_path  = os.path.join(data_path,'bace_chembl_common.csv')
pd.read_csv(common_path).set_index('smiles')[['Class']].to_csv(common_path + '.dmpnn.csv') #it needs smiles col is the first column
common_path = common_path + '.dmpnn.csv'


vl_pred_path = os.path.join(work_path,'DMPNN_pred_val.csv')
ts_pred_path = os.path.join(work_path,'DMPNN_pred_test.csv')
drug_pred_path = os.path.join(work_path,'DMPNN_pred_drug.csv')
novel_pred_path = os.path.join(work_path,'DMPNN_pred_novel.csv')
common_pred_path = os.path.join(work_path,'DMPNN_pred_common.csv')

task_name = "BACE"
task_type = "classification"


cf_path = os.path.join(work_path, "dmpnn_config")
model_save_path = os.path.join(work_path, "dmpnn_model_checkpoints")




# HPs
cmd_opt_params = "%s %s --data_path %s --dataset_type %s --num_iters 3 --config_save_path %s" % (pythoner, optimizer, 
                                                                                                  tr_path, task_type,
                                                                                                  cf_path)


# train, valid is used for find a best model
cmd_train = "%s %s --data_path %s --dataset_type %s --config_path %s --epochs 100 --save_dir %s --separate_val_path %s" % (pythoner,  
                                                                                                              trainer, 
                                                                                                              tr_path, 
                                                                                                              task_type, 
                                                                                                              cf_path, 
                                                                                                              model_save_path, 
                                                                                                              vl_path)
# predict on valid
cmd_predict_valid = "%s %s --test_path %s --checkpoint_dir %s --preds_path %s" % (pythoner, predicter, 
                                                                                  vl_path, model_save_path, 
                                                                                  vl_pred_path)

# predict on test
cmd_predict_test  = "%s %s --test_path %s --checkpoint_dir %s --preds_path %s" % (pythoner, predicter, 
                                                                                  ts_path, model_save_path,
                                                                                  ts_pred_path)

# predict on drug
cmd_predict_drug  = "%s %s --test_path %s --checkpoint_dir %s --preds_path %s" % (pythoner, predicter, 
                                                                                  drug_path, model_save_path,
                                                                                  drug_pred_path)

# predict on novel
cmd_predict_novel  = "%s %s --test_path %s --checkpoint_dir %s --preds_path %s" % (pythoner, predicter, 
                                                                                  novel_path, model_save_path,
                                                                                  novel_pred_path)

# predict on common
cmd_predict_common  = "%s %s --test_path %s --checkpoint_dir %s --preds_path %s" % (pythoner, predicter, 
                                                                                  common_path, model_save_path,
                                                                                  common_pred_path)


#tuning best parameters
os.system(cmd_opt_params)

#using best parameters to train, and using valid set to make an early stopping
os.system(cmd_train)

# make prediction
os.system(cmd_predict_valid)
os.system(cmd_predict_test)
os.system(cmd_predict_drug)
os.system(cmd_predict_novel)
os.system(cmd_predict_common)

0